[Документация](https://docs.python.org/3/library/asyncio.html#module-asyncio)

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [2]:
sleep_time = 1  # sec

## Заметки

- **coroutine** - функция с async, которая создаёт coroutine object и исполняется только при:
    - await
    - когда её оборачивают в Task
- **Task** - обёртка над корутиной для планирования её выполнения циклом событий
- **Event loop** - планирует задачи, обрабатывает I/O, колбэки и переключается при await

**asyncio.run(coro)**: создать цикл, выполнить корутину, закрыть цикл.\
**asyncio.create_task(coro)**: запланировать корутину как задачу, вернуть Task.\
**await asyncio.sleep(n)**: Отпускает поток ("Уступает" циклу событий).\
**await gather()**: запускает await сразу для нескольких корутин.

- Пока await из корутины не разрешился, выполнение кода продолжается в цикле событий.
- При завершении корутины, на место await подставляется результат.

## sync vs async (IO-Bound)

### sync

In [15]:
from time import sleep


def get_some_data():
    print('⌛ Получаем данные...')

    sleep(sleep_time)

    print('✅ Данные получены.')
    return {'status': 'ok', 'data': {'key': 'value'}}

def get_movies(): 
    print('⌛ Получаем список фильмов...')

    sleep(sleep_time)

    print('✅ Список фильмов получен.')
    return ['movie1', 'movie2', 'movie3']

In [16]:
import time

start_time = time.time()
data       = get_some_data()
movies     = get_movies()
end_time   = time.time()
print(f'Время выполнения (sync): {end_time - start_time:.2f} s')

⌛ Получаем данные...
✅ Данные получены.
⌛ Получаем список фильмов...
✅ Список фильмов получен.
Время выполнения (sync): 2.00 s


### async

In [17]:
async def async_get_some_data():
    print('⌛ Получаем данные...')

    await asyncio.sleep(sleep_time)

    print('✅ Данные получены.')
    return {'status': 'ok', 'data': {'key': 'value'}}

async def async_get_movies(): 
    print('⌛ Получаем список фильмов...')

    await asyncio.sleep(sleep_time)

    print('✅ Список фильмов получен.')
    return ['movie1', 'movie2', 'movie3']

In [ ]:
import time

start_time = time.time()

# await для двух корутин сразу
data, movies = await asyncio.gather(async_get_some_data(), async_get_movies())
end_time = time.time()

print(f'Время выполнения (async): {end_time - start_time:.2f} s')

⌛ Получаем данные...
⌛ Получаем список фильмов...
✅ Данные получены.
✅ Список фильмов получен.
Время выполнения (async): 1.00 s


## TaskGroup

- Объект задачи содержит внутри себя корутину (`Task(coro=coro, ...)`).
- Задачи внутри TaskGroup запускаются либо при выходе из async with, либо при первом await.

In [19]:
start_time = time.time()

async with asyncio.TaskGroup() as tg: 
    print('Запуск TaskGroup()...')
    
    data_task = tg.create_task(async_get_some_data())
    print('📥 Добавлена задача: get_some_data()')

    movies_task = tg.create_task(async_get_movies())
    print('📥 Добавлена задача: get_movies()')

print('✅ TaskGroup() выполнена')
end_time = time.time()
print(f'Время выполнения (async): {end_time - start_time:.2f} s')
    
print(data_task.result())
print(movies_task.result())


Запуск TaskGroup()...
📥 Добавлена задача: get_some_data()
📥 Добавлена задача: get_movies()
⌛ Получаем данные...
⌛ Получаем список фильмов...
✅ Данные получены.
✅ Список фильмов получен.
✅ TaskGroup() выполнена
Время выполнения (async): 1.00 s
{'status': 'ok', 'data': {'key': 'value'}}
['movie1', 'movie2', 'movie3']


## timeout

In [24]:
async def foo(): 
    print('🚀 Запуск foo()...')
    await asyncio.sleep(2)
    print('✅ foo() вернула значение')
    return 'foo'

async def bar(): 
    print('🚀 Запуск bar()...')
    await asyncio.sleep(1)
    print('✅ bar() вернула значение')
    return 'bar'

async def get_something(): 
    
    print('⚙️ Запуск get_something()...')
    
    async with asyncio.timeout(2): 
        foo_result, bar_result = await asyncio.gather(foo(), bar())

    print(foo_result)
    print(bar_result)

try:
    await get_something()
except TimeoutError: 
    print('TimeoutError')

⚙️ Запуск get_something()...
🚀 Запуск foo()...
🚀 Запуск bar()...
✅ bar() вернула значение
TimeoutError


## CPU-bound задачи внутри async
CPU-bound задачи (или синхронный sleep) внутри корутин блокируют поток и не дают идти циклу событий.

In [ ]:
async def func1(): 
    print('🚀 Запуск func1()...')
    for i in range(10_000):   # ~ 2s 
        
        # Переключаем контекст для того, что бы дать пройти циклу событий
        if i % 100 == 0:
            await asyncio.sleep(0)
        # Иначе поток заблокируется
        i ** i

    print('✅ func1(): OK')   
    return 'func1'

async def func2(): 
    print('🚀 Запуск func2()...')
    await asyncio.sleep(1)
    print('✅ func2(): OK')   
    return 'func2'

async def run_f1_f2(): 
    start_time = time.time()

    async with asyncio.timeout(2): 
        f1, f2 = await asyncio.gather(func1(), func2())
    
    end_time = time.time()

    print(f'Время выполнения (async): {end_time - start_time:.2f} s')

try:
    asyncio.run(run_f1_f2())
except TimeoutError: 
    print('TimeoutError')

🚀 Запуск func1()...
🚀 Запуск func2()...
✅ func2(): OK
TimeoutError
